In [1]:
import os
import re
import glob
import xarray as xr
import numpy as np
from pathlib import Path
from typing import Union, Optional, Callable
from tqdm import tqdm
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster, progress

In [2]:
def find_years_and_variables(list_files: list[str]) -> tuple[list[str], list[str]]: 
    years = set()
    variables = set()
    
    for file in list_files:
        # Extract the year (4 digits) and variable name (anything before .nc after the coordinates)
        match = re.search(r'/(\d{4})-\d{2}_\[.*?\]_(.*)\.nc$', file)
        if match:
            year = match.group(1)
            variable = match.group(2)
            
            # Add to sets to ensure uniqueness
            years.add(year)
            variables.add(variable)
    
    # Convert sets to sorted lists
    years = sorted(list(years))
    variables = sorted(list(variables))

    return years, variables


def select_data_common_year_variable(year: str, variable: str, list_files: list[str]) -> list[str]:

    filter_conditions = rf"(?=.*{year}-0[1-9]|.*{year}-1[0-2])(?=.*{variable})"
    
    return [file for file in list_files if re.search(filter_conditions,file)]


def preprocess_drop_coord(ds: xr.Dataset, coord_to_drop: str|list[str] = ["expver", "surface"]):
    return ds.drop_vars(coord_to_drop, errors='ignore')


def convert_nc_to_zarr(file_list, output_zarr_path, chunk_dict: Union[dict, str] = "auto", preprocess: Optional[Callable]=None):
    
    ds = xr.open_mfdataset(file_list, 
                           combine='by_coords',
                           engine="h5netcdf",
                           parallel=True,
                           # decode_cf=False,
                           preprocess=preprocess,
                           coords="minimal",
                          ).chunk(chunk_dict)

    with ProgressBar():
        ds.to_zarr(output_zarr_path, mode='w', consolidated=True)

@dask.delayed
def save_zarr_dask(list_files: list[str], output_file: str, chunk_dict: Union[dict, str] = "auto", preprocess: Optional[Callable]=None): 
    try:
        if os.path.exists(output_file):
            print(f"path already exists, skipping: {output_zarr_path}")
        else:
            convert_nc_to_zarr(list_files, output_file, chunk_dict, preprocess=preprocess)
    except Exception as e:
        print(f"error!: \n{e}")
        

In [3]:
data_dir = "/scratch/opodriscoll/data/ERA5"

list_files = glob.glob(str(data_dir) + "/*.nc")

dict_chunk = {
    "valid_time": 200,
    "latitude": 200,
    "longitude": 200,
}

years, variables = find_years_and_variables(list_files)

In [5]:
with LocalCluster() as cluster, Client(cluster) as client:
    
    outputs = []
    
    for year in years:
        for variable in variables:
    
            data_str_filt = select_data_common_year_variable(year=year, variable=variable, list_files=list_files)
                    
            output_zarr_path = f"/scratch/opodriscoll/data/ERA5/{year}_{variable}.zarr"
    
            output = save_zarr_dask(
                list_files=data_str_filt,
                output_file=output_zarr_path,
                chunk_dict=dict_chunk,
                preprocess=preprocess_drop_coord
            )
            outputs.append(output)

    dask.compute(outputs[:4])
            

## Confirm successful storage 

In [1]:
list_files = glob.glob(str(data_dir) + "/*.zarr")
list_files.sort()
list_files

NameError: name 'glob' is not defined

In [4]:
list_files = glob.glob(str(data_dir) + "/*.zarr")
list_files.sort()
list_files

['/scratch/opodriscoll/data/ERA5/2020_10m_u_component_of_wind.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_10m_v_component_of_wind.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_2m_dewpoint_temperature.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_2m_temperature.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_boundary_layer_heighttotal_precipitation.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_sea_level_pressure.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_surface_downward_long_wave_radiation_flux.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_surface_downward_short_wave_radiation_flux.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_surface_latent_heat_flux.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_surface_sensible_heat_flux.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_mean_wave_period.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_ocean_surface_stress_equivalent_10m_neutral_wind_direction.zarr',
 '/scratch/opodriscoll/data/ERA5/2020_ocean_surface_stress_equivalent_10m_n

In [6]:
t = xr.open_mfdataset(list_files, engine="zarr", preprocess=preprocess_drop_coord)

In [7]:
t

<xarray.Dataset> Size: 907GB
Dimensions:     (latitude: 561, longitude: 1440, valid_time: 17544)
Coordinates:
  * latitude    (latitude) float64 4kB -70.0 -69.75 -69.5 ... 69.5 69.75 70.0
  * longitude   (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * valid_time  (valid_time) datetime64[ns] 140kB 2020-01-01 ... 2021-12-31T2...
    number      int64 8B 0
Data variables: (12/16)
    blh         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    dwi         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 140, 180), meta=np.ndarray>
    msdwlwrf    (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    msdwswrf    (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    msl         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    ...          ...
    sst         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    swh         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 140, 180), meta=np.ndarray>
    t2m         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 187, 200), meta=np.ndarray>
    wind        (valid_time, latitude, longitude) float32 57GB dask.array<chunksize=(200, 140, 180), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2024-10-09T20:37 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [23]:
n_time = t.sizes['valid_time']
n_lat = t.sizes['latitude']
n_lon = t.sizes['longitude']

# Generate random indices to sample from
n_samples = 100000
random_time_idx = np.random.randint(0, n_time, n_samples)
random_lat_idx = np.random.randint(0, n_lat, n_samples)
random_lon_idx = np.random.randint(0, n_lon, n_samples)

# Get the corresponding coordinate values for the random samples
random_valid_times = t['valid_time'].values[random_time_idx]
random_latitudes = t['latitude'].values[random_lat_idx]
random_longitudes = t['longitude'].values[random_lon_idx]

In [25]:
p = t.sel(
    valid_time=xr.DataArray(random_valid_times, dims='points'),
    latitude=xr.DataArray(random_latitudes, dims='points'),
    longitude=xr.DataArray(random_longitudes, dims='points'),
    method='nearest'
).compute()
p

<xarray.Dataset> Size: 9MB
Dimensions:     (points: 100000)
Coordinates:
    latitude    (points) float64 800kB 24.0 -10.25 5.25 ... 47.5 -62.5 15.75
    longitude   (points) float64 800kB -94.5 -28.5 117.2 ... 106.5 141.0 2.5
    valid_time  (points) datetime64[ns] 800kB 2020-02-16T23:00:00 ... 2020-06...
    number      int64 8B 0
    surface     float64 8B 0.0
Dimensions without coordinates: points
Data variables: (12/16)
    blh         (points) float32 400kB 311.2 942.3 353.4 ... 1.006e+03 35.77
    d2m         (points) float32 400kB 295.7 296.5 297.8 ... 262.8 263.2 287.1
    dwi         (points) float32 400kB 110.5 nan nan nan ... nan nan 216.6 nan
    msdwlwrf    (points) float32 400kB 384.4 401.0 421.5 ... 267.4 223.9 420.4
    msdwswrf    (points) float32 400kB 292.2 314.3 366.0 0.0 ... 6.438 0.0 0.0
    msl         (points) float32 400kB 1.014e+05 1.012e+05 ... 1.009e+05
    ...          ...
    sst         (points) float32 400kB 296.5 301.0 nan 297.9 ... nan 272.6 nan
    swh         (points) float32 400kB 1.017 nan nan nan ... nan nan 2.536 nan
    t2m         (points) float32 400kB 297.0 300.0 301.1 ... 269.2 268.5 307.9
    u10         (points) float32 400kB -2.929 -7.781 0.6748 ... 4.787 0.7857
    v10         (points) float32 400kB 1.185 2.675 0.02884 ... 6.898 -1.389
    wind        (points) float32 400kB 2.846 nan nan nan ... nan nan 8.375 nan
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2024-10-09T20:37 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts